<a href="https://colab.research.google.com/github/ayzandam/DeepLearning-Tensorflow/blob/main/Vision_with_ImageGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The training data

!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2023-11-30 14:51:32--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.197.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   164MB/s    in 0.9s    

2023-11-30 14:51:33 (164 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [4]:
import os

# Define directories
train_horse_dir = os.path.join('./horse_or_human/horses')
train_human_dir = os.path.join('./horse_or_human/humans')

In [5]:
import tensorflow as tf

# Define the model
model = tf.keras.Sequential([
    # The First Convolution
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The Second Convolution
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The Third Convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The Fourth Convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The Fifth Convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Flatten the output
    tf.keras.layers.Flatten(),
    # The 512-Neurons Deep Hidden Layers
    tf.keras.layers.Dense(512, activation='relu'),
    # The output layer
    tf.keras.layers.Dense(1, activation='sigmoid')

])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [7]:
from tensorflow.keras.optimizers import RMSprop

# Model Compilation
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Image Preprocessing using ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescale all images
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory('./horse-or-human/',
                                                    target_size=(300, 300),
                                                    batch_size=128,
                                                    class_mode='binary')

Found 1027 images belonging to 2 classes.


In [10]:
# Train the Model
history = model.fit(train_generator,
                    steps_per_epoch=8,
                    epochs=15,
                    verbose=1)

Epoch 1/15
8/8 [==============================] - 7s 851ms/step - loss: 0.0924 - accuracy: 0.9644
Epoch 2/15
8/8 [==============================] - 7s 876ms/step - loss: 0.0604 - accuracy: 0.9789
Epoch 3/15
8/8 [==============================] - 10s 1s/step - loss: 0.3069 - accuracy: 0.9032
Epoch 4/15
8/8 [==============================] - 7s 838ms/step - loss: 0.0817 - accuracy: 0.9867
Epoch 5/15
8/8 [==============================] - 8s 928ms/step - loss: 0.0278 - accuracy: 0.9922
Epoch 6/15
8/8 [==============================] - 6s 777ms/step - loss: 0.0223 - accuracy: 0.9922
Epoch 7/15
8/8 [==============================] - 7s 758ms/step - loss: 0.0079 - accuracy: 0.9989
Epoch 8/15
8/8 [==============================] - 8s 928ms/step - loss: 0.1205 - accuracy: 0.9689
Epoch 9/15
8/8 [==============================] - 6s 784ms/step - loss: 0.0120 - accuracy: 0.9978
Epoch 10/15
8/8 [==============================] - 7s 781ms/step - loss: 0.2371 - accuracy: 0.9277
Epoch 11/15
8/8 [====

In [ ]:
## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
